# Eternal wall
 - Bitcoin transactions support adding small data in each transaction
 - Used by spenders to add meta information viz ticket #
 - Data will be stored by all nodes (forever)


# Imports
- Import python libraries for constructing and signing transactions

In [ ]:
# imports
import sys
from bitcointx.wallet import CCoinKey, P2PKHCoinAddress
from bitcointx.core.key import (
    CPubKey, CKeyBase, CExtKeyBase, CExtPubKeyBase
)
from bitcointx.core import CMutableTxIn, CMutableTxOut, CMutableTransaction, COutPoint, CTxInWitness, CTxWitness, b2lx, x, lx, b2x, Hash, Hash160, CTransaction, CoreCoinParams, CTxOut
from bitcointx.core.script import CScript, CScriptWitness, OP_CHECKSIG, SignatureHash, SIGHASH_ALL, SIGVERSION_WITNESS_V0, OP_RETURN
from bitcointx.wallet import CBitcoinSecret, P2PKHBitcoinAddress, P2WPKHBitcoinAddress, CBitcoinAddress
import binascii
from bitcointx.core.scripteval import VerifyScript
from bitcointx.signmessage import BitcoinMessage, VerifyMessage, SignMessage
import os

# Create new account
- Easy to create accounts on fly
- General recommendation is to use new account for each transaction
- Hardware wallets like ledger nano, trezor

In [ ]:
privkey = CBitcoinSecret.from_secret_bytes(os.urandom(32))

In [ ]:
# create new account
privkey = CBitcoinSecret.from_secret_bytes(os.urandom(32))
print("private key", privkey)
print("public key", P2PKHBitcoinAddress.from_pubkey(privkey.pub))

# Fund account

- Fund new account with some coins
- Spend these coins to pay for the transaction that writes message
- Recall that each transaction spends the coin in entirety
- Specify change address for  balance less the transaction fee

# Read message from input

In [ ]:
# read message from file
msgfile = "./message2"
msg = ""
with open(msgfile, 'rb') as f:
    msg = f.read(40)
print("bytes ", msg)

In [ ]:
msg = input("any message upto 20 characters")
msg = str.encode(msg)
print(f"Message is {msg}")

In [ ]:
# secrets
#privkey = CBitcoinSecret(
    "1VsqF24jNusMxaSx6o7AF14g4yEwzGoCE")

# Transaction input 
- Specify hash of the transaction we wish to spend

In [ ]:
# spending output aka input for the transaction
#txid = lx('3c738db4f8c32a6e150e0525b79e836a1509ee67543a353840ab456d0512db6a')
txid = lx('f381136682b13d7834c219d4bb5e31b827d3e0ffacd30da718d7396ad3be6126')
COIN = 100000000
vout = 0
value_in = 0.00010000 * COIN

In [ ]:
# transaction skeleton
txin = CMutableTxIn(COutPoint(txid, vout))
# script pub key for the spend utxo
txin_scriptPubKey = P2PKHBitcoinAddress.from_pubkey(
    privkey.pub).to_scriptPubKey()
print(txin_scriptPubKey.hex())

# Specify outputs
- One output is for the change
- Other (unspendable) output is for the message

In [ ]:
# create change transaction and message transaction
msgout = CMutableTxOut(0, CScript([OP_RETURN, msg]))
txout = CMutableTxOut(
    5000,
    CBitcoinAddress('1VsqF24jNusMxaSx6o7AF14g4yEwzGoCE').to_scriptPubKey())

# Fee
- Add some fee for the miner

In [ ]:
# create transaction with input and output
tx = CTransaction([txin], [txout, msgout]).to_mutable()
# add fee for miner
FEE_PER_BYTE = 0.00005 * COIN / 1000
tx.vout[0].nValue = int(
    value_in - max(len(tx.serialize()) * FEE_PER_BYTE, 0.00005 * COIN))

# Sign

- previous transaction is signed to prove ownership of coins
- Signature can't be forged, irrefutable ownership

In [ ]:
# get hash for signing
sighash = SignatureHash(txin_scriptPubKey, tx, 0, SIGHASH_ALL)
# Now sign it. We have to append the type of signature we want to the end, in
# this case the usual SIGHASH_ALL.
sig = privkey.sign(sighash) + bytes([SIGHASH_ALL])
print(sig.hex())

In [ ]:
# add signature to the scriptSig
txin.scriptSig = CScript([sig, privkey.pub])
tx = CTransaction([txin], [txout, msgout])

In [ ]:
# print rawtransaction and size of transaction
tx_bytes = tx.serialize()
tx_hex = b2x(tx_bytes)
print(tx_hex)
print(len(tx_bytes), 'bytes', file=sys.stderr)

# Publish transaction to the network

- Everything until now is done offline
- We can now publish the transaction to any Bitcoin node
- Gossip protocol ensures that the transaction eventually reaches all nodes

# Let's check the message from last wintersession
- https://blockstream.info/tx/ee73a7a5e2bbb7b981a30398343e2c6ff50468060aa172f2f04a883fb1200fc1
- Message is secured by > 51k bitcoin blocks